<a href="https://colab.research.google.com/github/hasanocal7/Kur_3/blob/main/CatBoost_%26_Random_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

In [63]:
import numpy as np
import pandas as pd

from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/pooja2512/Adult-Census-Income/master/adult.csv")
data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [ ]:
columns_with_q_mark = [col for col in data.columns if (data[col] == '?').any()]

In [ ]:
for col in columns_with_q_mark:
  data[col] = data[col].replace('?', 'Unknown')

data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,Unknown,77053,HS-grad,9,Widowed,Unknown,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,Unknown,186061,Some-college,10,Widowed,Unknown,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [ ]:
data['income'].value_counts(normalize = True) * 100

<=50K    75.919044
>50K     24.080956
Name: income, dtype: float64

In [ ]:
X, y = data.drop(['income'], axis = 1), data['income']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2,
                                                    random_state=34589,
                                                    stratify=y)

In [ ]:
categorical_features = list(X_train.select_dtypes(include = 'object').columns)

In [ ]:
train_pool = Pool(X_train, y_train, cat_features = categorical_features)
test_pool = Pool(X_test, y_test, cat_features = categorical_features)

In [ ]:
"""
XGBoost, LightGBM
CatBoost --> Ordering Boosting diye bir method kullanılır.
            - Overfittinge karşı bir tık dayanıklı.

            Overfitting (Aşırı Uyum): ML modelimizin train ettiğimiz verisetinde
            çok iyi performans gösterip, hiç görmediği test veya gelecek olan
            verisetinde kötü performans göstermesidir. Modelimizin bir nevi
            train verisetini ezberlemesidir.

            Bizim ML'deki amacımız train verisetini genellemektedir.
"""
model = CatBoostClassifier(n_estimators = 300,
                           task_type = 'GPU')

In [ ]:
param_dist ={
    'learning_rate': [0.03, 0.1, 0.3],
    'depth': [3, 5, 7],
    'l2_leaf_reg' : [1, 3, 5] # l2_leaf_regularization
}

In [ ]:
search_results = model.randomized_search(param_dist, train_pool, cv = 3, n_iter = 10, partition_random_seed=54784, verbose = 1)

0:	learn: 0.4802264	test: 0.4776736	best: 0.4776736 (0)	total: 27.7ms	remaining: 8.28s
1:	learn: 0.4091329	test: 0.4083557	best: 0.4083557 (1)	total: 61.4ms	remaining: 9.15s
2:	learn: 0.3750300	test: 0.3739399	best: 0.3739399 (2)	total: 86.7ms	remaining: 8.58s
3:	learn: 0.3510537	test: 0.3517298	best: 0.3517298 (3)	total: 112ms	remaining: 8.29s
4:	learn: 0.3395033	test: 0.3411228	best: 0.3411228 (4)	total: 138ms	remaining: 8.16s
5:	learn: 0.3333688	test: 0.3357222	best: 0.3357222 (5)	total: 179ms	remaining: 8.78s
6:	learn: 0.3295182	test: 0.3326199	best: 0.3326199 (6)	total: 204ms	remaining: 8.53s
7:	learn: 0.3243788	test: 0.3284007	best: 0.3284007 (7)	total: 234ms	remaining: 8.55s
8:	learn: 0.3198795	test: 0.3238033	best: 0.3238033 (8)	total: 260ms	remaining: 8.41s
9:	learn: 0.3156989	test: 0.3199174	best: 0.3199174 (9)	total: 286ms	remaining: 8.29s
10:	learn: 0.3138217	test: 0.3187662	best: 0.3187662 (10)	total: 312ms	remaining: 8.21s
11:	learn: 0.3122874	test: 0.3176389	best: 0.3176

In [ ]:
search_results['params']

{'depth': 7, 'l2_leaf_reg': 3, 'learning_rate': 0.3}

In [56]:
y_pred = model.predict(test_pool)

In [59]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       <=50K       0.89      0.94      0.91      4945
        >50K       0.76      0.65      0.70      1568

    accuracy                           0.87      6513
   macro avg       0.83      0.79      0.81      6513
weighted avg       0.86      0.87      0.86      6513



In [64]:
accuracy_score(y_test, y_pred)

0.8671886995240289